## This notebook runs the setup for JBrowse2 using Bash commands.

### Instructions for Running the Notebook:

1. Ensure you have created the APACHE_ROOT environment variable pointing to your Apache root directory.
   Example: export APACHE_ROOT='/path/to/apache/root'
   If you're unsure of the correct path, you can find it by running:
   sudo find / -name "www" 2>/dev/null

2. Ensure the required dependencies are installed, such as wget, samtools, bowtie2, and jbrowse.

3. Modify the APACHE_ROOT and WORKDIR variables in the notebook to reflect your setup.

4. Execute the notebook cells in order.

Notes:
- The notebook assumes you have a JBrowse2 instance installed in the $APACHE_ROOT/jbrowse2 directory.
- If any errors occur, the notebook will output the error message and stop execution.


### Error Handling Function and Setting up Directories

In [24]:
%%bash
set -e  # Stop execution on errors

# Define WORKDIR and APACHE_ROOT
WORKDIR="/Users/katebarouch/tmp"  # Writable directory
mkdir -p "$WORKDIR"
chmod u+w "$WORKDIR"
cd "$WORKDIR"

APACHE_ROOT="/opt/homebrew/var/www"

echo "Working directory: $WORKDIR"
echo "Apache root directory: $APACHE_ROOT"

# Ensure Apache root exists
if [ ! -d "$APACHE_ROOT/jbrowse2" ]; then
    echo "Error: JBrowse2 not found at $APACHE_ROOT/jbrowse2. Please check your APACHE_ROOT variable."
    exit 1
fi

Working directory: /Users/katebarouch/tmp
Apache root directory: /opt/homebrew/var/www


### Download and Process Dengue Referencec Genome

In [26]:
%%bash
set -e  # Stop execution on errors

cd "/Users/katebarouch/tmp"  # Ensure you're in the working directory

echo "Downloading Dengue genome (reference genome)..."
wget -q "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/862/125/GCF_000862125.1_ViralProj15306/GCF_000862125.1_ViralProj15306_genomic.fna.gz" -O genome.fna.gz

echo "Decompressing reference genome file..."
gunzip -f genome.fna.gz || exit 1

echo "Renaming reference genome file..."
mv genome.fna viral_genome.fa || exit 1

echo "Indexing reference genome file with samtools..."
samtools faidx viral_genome.fa || exit 1

Decompressing reference genome file...
Renaming reference genome file...
Indexing reference genome file with samtools...


### Add Reference Genome to JBrowse

In [31]:
%%bash

set -e

cd "/Users/katebarouch/tmp"  

APACHE_ROOT="/opt/homebrew/var/www"

echo "Adding reference genome assembly to JBrowse..."
jbrowse add-assembly viral_genome.fa --out "$APACHE_ROOT/jbrowse2" --load copy || exit 1


viral_genome.fa
viral_genome.fa.fai
Adding reference genome assembly to JBrowse...
Added assembly "viral_genome" to /opt/homebrew/var/www/jbrowse2/config.json


### Download and Process Annotations

In [32]:
%%bash

set -e

cd "/Users/katebarouch/tmp"  


APACHE_ROOT="/opt/homebrew/var/www"

echo "Downloading genome annotations..."
wget -q "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/862/125/GCF_000862125.1_ViralProj15306/GCF_000862125.1_ViralProj15306_genomic.gff.gz" -O annotations.gff.gz || exit 1

echo "Decompressing annotations file..."
gunzip -f annotations.gff.gz || exit 1

echo "Sorting GFF3 annotations..."
jbrowse sort-gff annotations.gff > genes.gff || exit 1

echo "Compressing sorted GFF3 file..."
bgzip -f genes.gff || exit 1

echo "Indexing compressed GFF3 file with tabix..."
tabix genes.gff.gz || exit 1

echo "Adding annotations track to JBrowse..."
jbrowse add-track genes.gff.gz --out "$APACHE_ROOT/jbrowse2" --load copy || exit 1



Decompressing annotations file...
Sorting GFF3 annotations...
Compressing sorted GFF3 file...
Indexing compressed GFF3 file with tabix...
Adding annotations track to JBrowse...
Added track with name "genes.gff" and trackId "genes.gff" to /opt/homebrew/var/www/jbrowse2/config.json
